<a href="https://colab.research.google.com/github/quynhhuongngng/ShecodesHackathon2021/blob/main/ConnectMe_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Library

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# 2. Load data

In [ ]:
!gdown https://drive.google.com/uc?id=1sDbn2nK8iDv2COKuXPyWVgwrYfAwXtNp
!gdown https://drive.google.com/uc?id=1zHVGLDLSbBSveXcHHDu5CV_cAtEneMPM
!gdown https://drive.google.com/uc?id=1evDdPDCJ13bp0IpQ4VQVRFQtYDo0l9fq
!gdown https://drive.google.com/uc?id=1ZOQPIxscTc74yGrMldRvTr-RD69DL8Hu

Downloading...
From: https://drive.google.com/uc?id=1YMAOthfEkpoN5gSnMNZKrFmIiIKCxhO-
To: /content/HKK.csv
100% 357/357 [00:00<00:00, 658kB/s]
Downloading...
From: https://drive.google.com/uc?id=1zHVGLDLSbBSveXcHHDu5CV_cAtEneMPM
To: /content/MTQ.csv
100% 181/181 [00:00<00:00, 309kB/s]
Downloading...
From: https://drive.google.com/uc?id=1O6h2iY_8rx0nXM_RjBvDlW2vkbiZN3Z0
To: /content/Rating.csv
100% 802/802 [00:00<00:00, 1.32MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZOQPIxscTc74yGrMldRvTr-RD69DL8Hu
To: /content/VP.csv
100% 115/115 [00:00<00:00, 103kB/s]


In [ ]:
df1 = pd.read_csv('/content/Rating.csv', dtype=str)
df2 = pd.read_csv('/content/HKK.csv', dtype=str)
df3 = pd.read_csv('/content/MTQ.csv', dtype=str)

In [ ]:
df1.head()

,ID_HKK,ID_MTQ,Rating
0,00000000001,5900000001,5
1,00000000002,5900000001,4
2,00000000003,5900000001,4
3,00000000004,5900000003,5
4,00000000005,5900000005,5


In [ ]:
df1['Rating'] = df1['Rating'].astype('uint8')

In [ ]:
df1.dtypes

ID_HKK    object
ID_MTQ    object
Rating     uint8
dtype: object

In [ ]:
df2.head()

,ID_HKK,VATPHAM
0,00000000001,VP01
1,00000000002,VP01
2,00000000003,VP01
3,00000000004,VP02
4,00000000005,VP01


In [ ]:
df3.head()

,ID_MTQ,VATPHAM
0,5900000001,VP01
1,5900000002,VP01
2,5900000003,VP02
3,5900000004,VP03
4,5900000005,VP01


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID_HKK  25 non-null     object
 1   ID_MTQ  25 non-null     object
 2   Rating  25 non-null     uint8 
dtypes: object(2), uint8(1)
memory usage: 553.0+ bytes


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID_HKK   16 non-null     object
 1   VATPHAM  16 non-null     object
dtypes: object(2)
memory usage: 384.0+ bytes


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID_MTQ   6 non-null      object
 1   VATPHAM  6 non-null      object
dtypes: object(2)
memory usage: 336.0+ bytes


# Recommendor 1 : Demographic Filtering

1. Top các trường hợp khó khăn đáng chú ý nhất

In [ ]:
data = df1.groupby('ID_HKK').agg(['mean', 'count'])['Rating'].reset_index()
# tạo điểm dựa trên xếp hạng trung bình và rating của mạnh thường quân được xếp hạng
m = data['count'].quantile(0.5) # phiếu bầu tối thiểu cần thiết để được liệt kê trong Top 250
data = data[data['count']>m]
print('m =', m)
print(data.shape)
R = data['mean'] # Trung bình rating
v = data['count'] # Tổng số rating
C = data['mean'].mean() 
data['weighted rating'] = (v/(v+m))*R + (m/(v+m))*C
data = data.sort_values('weighted rating', ascending=False).reset_index(drop=True)


data = pd.merge(data, df2, on='ID_HKK')[['ID_HKK', 'VATPHAM', 'mean', 'count', 'weighted rating', 
                              ]].drop_duplicates('ID_HKK')
data

m = 1.5
(8, 3)


,ID_HKK,VATPHAM,mean,count,weighted rating
0,00000000004,VP02,5.000000,2,4.767857
1,00000000014,VP01,5.000000,2,4.767857
2,00000000006,VP01,4.666667,3,4.597222
3,00000000001,VP01,4.500000,2,4.482143
4,00000000002,VP01,4.500000,2,4.482143
5,00000000005,VP01,4.500000,2,4.482143
6,00000000007,VP01,4.000000,2,4.196429
7,00000000008,VP02,3.500000,2,3.910714


2. Top các nhà mạnh thường quân được tương tác mạnh nhất

In [ ]:
data = df1.groupby('ID_MTQ').agg(['mean', 'count'])['Rating'].reset_index()
# tạo điểm dựa trên xếp hạng trung bình và rating của mạnh thường quân được xếp hạng
m = data['count'].quantile(0.5) # phiếu bầu tối thiểu cần thiết để được liệt kê trong Top 250
data = data[data['count']>m]
print('m =', m)
print(data.shape)
R = data['mean'] # Trung bình rating
v = data['count'] # Tổng số rating
C = data['mean'].mean() 
data['weighted rating'] = (v/(v+m))*R + (m/(v+m))*C
data = data.sort_values('weighted rating', ascending=False).reset_index(drop=True)


data = pd.merge(data, df3, on='ID_MTQ')[['ID_MTQ', 'VATPHAM', 'mean', 'count', 'weighted rating', 
                              ]].drop_duplicates('ID_MTQ')
data


m = 5.0
(2, 3)


,ID_MTQ,VATPHAM,mean,count,weighted rating
0,5900000005,VP01,4.625000,8,4.614698
1,5900000001,VP01,4.571429,7,4.582589


# Recommender 2: Collaborative Filtering

In [ ]:
# merge df1 and df5 to get movie titles and drop rows for which title is not available
data = pd.merge(df1, df3, on='ID_MTQ')

# get total counts of no. of occurence of book
data['count'] = data.groupby('ID_MTQ').transform('count')['ID_HKK']

# fetch top 100 books based on count
id_mtq = data.drop_duplicates('ID_MTQ').sort_values(
    'count', ascending=False).iloc[:100]['ID_MTQ']

# filter out data as per the ISBN
data = data[data['ID_MTQ'].isin(id_mtq)].reset_index(drop=True)
data

,ID_HKK,ID_MTQ,Rating,VATPHAM,count
0,00000000001,5900000001,5,VP01,7
1,00000000002,5900000001,4,VP01,7
2,00000000003,5900000001,4,VP01,7
3,00000000014,5900000001,5,VP01,7
4,00000000004,5900000001,5,VP01,7
5,00000000005,5900000001,4,VP01,7
6,00000000006,5900000001,5,VP01,7
7,00000000004,5900000003,5,VP02,5
8,00000000008,5900000003,3,VP02,5
9,00000000009,5900000003,5,VP02,5


In [ ]:
# tạo ma trận xếp hạng sách của MTQ
df = data.pivot(index='ID_HKK', columns='ID_MTQ', values='Rating')
print(df.head())
print(data.shape)

ID_MTQ       5900000001  5900000002  5900000003  5900000004  5900000005
ID_HKK                                                                 
00000000001         5.0         NaN         NaN         NaN         4.0
00000000002         4.0         NaN         NaN         NaN         5.0
00000000003         4.0         NaN         NaN         NaN         NaN
00000000004         5.0         NaN         5.0         NaN         NaN
00000000005         4.0         NaN         NaN         NaN         5.0
(25, 5)


Using Surpise Library

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 50 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619422 sha256=35b8adccb0f69ce0952f226425fd789d64bac44d44e09d02836bb8db514f6bf3
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, cross_validate

In [ ]:
reader = Reader(rating_scale=(0, 10))
surprise_data = Dataset.load_from_df(data[['ID_HKK', 'ID_MTQ', 'Rating']], reader)
trainset, testset = train_test_split(surprise_data, test_size=0.25)

In [ ]:
benchmark = []
# Lặp lại tất cả các thuật toán 
for algorithm in [SVD()]:
    # Thực hiện cross validation
    results = cross_validate(algorithm, surprise_data, measures=['RMSE'], cv=3, verbose=False)
    
    # Nhận kết quả và lưu trữ thuật toán
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.642912,0.0041,0.000163


In [ ]:
svd = SVD() 
svd.fit(trainset)

In [ ]:
index_val = 5
# get user id
userId = df.index[index_val]
books = []
ratings = []
titles = []

for isbn in df.iloc[index_val][df.iloc[index_val].isna()].index:
    books.append(isbn)
    title = data[data['ID_MTQ']==isbn]['ID_MTQ'].values[0]
    titles.append(title)
    ratings.append(svd.predict(userId, isbn).est)

prediction = pd.DataFrame({'ID_MTQ':books, 'Rating':ratings, 'ID_HKK':userId})  
prediction = prediction.sort_values('Rating', ascending=False).iloc[:10].reset_index(drop=True)

# get other high rated books by user
temp = data[data['ID_HKK']==df.index[index_val]].sort_values(
    'Rating', ascending=False)[['Rating','ID_MTQ', 'ID_HKK']].iloc[:10].reset_index(drop=True)
prediction['ID_MTQ'] = temp['ID_MTQ']
prediction['Rated']= temp['Rating']
prediction

,ID_MTQ,Rating,ID_HKK,Rated
0,5900000001,4.714156,00000000006,5
1,5900000003,4.623682,00000000006,5


In [ ]:
index_val = 3
# get user id
userId = df.index[index_val]
books = []
ratings = []
titles = []

for isbn in df.iloc[index_val][df.iloc[index_val].isna()].index:
    books.append(isbn)
    title = data[data['ID_MTQ']==isbn]['ID_MTQ'].values[0]
    titles.append(title)
    ratings.append(svd.predict(userId, isbn).est)

prediction = pd.DataFrame({'ID_MTQ':books, 'Rating':ratings, 'ID_HKK':userId})  
prediction = prediction.sort_values('Rating', ascending=False).iloc[:10].reset_index(drop=True)

# get other high rated books by user
temp = data[data['ID_HKK']==df.index[index_val]].sort_values(
    'Rating', ascending=False)[['Rating','ID_MTQ', 'ID_HKK']].iloc[:10].reset_index(drop=True)
prediction['ID_MTQ'] = temp['ID_MTQ']
prediction['Rated']= temp['Rating']
prediction

,ID_MTQ,Rating,ID_HKK,Rated
0,5900000001,4.800684,00000000004,5.0
1,5900000003,4.618516,00000000004,5.0
2,NaN,4.470172,00000000004,NaN


2. Using mean of other user' weighted ratings based on similarity matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# thay thế NaN bằng xếp hạng trung bình dựa trên người dùng trong khung dữ liệu tổng hợp
df_imputed = df.fillna(df.mean(axis=0))

# có được sự tương đồng giữa tất cả người dùng
similarity_matrix = cosine_similarity(df_imputed.values)

In [ ]:
def get_recommendation(user_index):
    idx = user_index
    sim_scores = list(enumerate(similarity_matrix[idx]))

    # nhận id những mạnh thường quân chưa được xếp hạng bởi người dùng nhất định
    unrated_MTQ = df.iloc[idx][df.iloc[idx].isna()].index

    # nhận xếp hạng có trọng số của sách chưa được xếp hạng bởi tất cả những người dùng khác
    MTQ_ratings = (df[unrated_MTQ].T * similarity_matrix[idx]).T

    # có được 100 người dùng tương tự hàng đầu bằng cách bỏ qua người dùng hiện tại
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:101]

    # nhận được trung bình của xếp hạng mạnh thường quân bởi 100 người dùng 
    # tương tự nhất hàng đầu cho những mạnh thường quân chưa được xếp hạng
    MTQ_ratings = MTQ_ratings.iloc[[x[0] for x in sim_scores]].mean()
    
    # loại bỏ các giá trị rỗng và sắp xếp nó dựa trên xếp hạng
    MTQ_ratings = MTQ_ratings.reset_index().dropna().sort_values(0, ascending=False).iloc[:10]
    
    # nhận các đầu sách được đề xuất theo thứ tự đã sắp xếp
    recommended_MTQ = data[data['ID_MTQ'].isin(MTQ_ratings['ID_MTQ'])][['ID_MTQ']]
    recommended_MTQ = recommended_MTQ.drop_duplicates('ID_MTQ').reset_index(drop=True)
    assumed_ratings = MTQ_ratings[0].reset_index(drop=True)

    return pd.DataFrame({'ID_MTQ':recommended_MTQ['ID_MTQ'], 
                         'Assumed Rating':assumed_ratings})

In [ ]:
user_index = 10
recommended_MTQ = get_recommendation(user_index)
# nhận được những mạnh thường quân được người dùng đánh giá cao khác
temp = data[data['ID_HKK']==df.index[user_index]].sort_values(
    'Rating', ascending=False)[['Rating', 'ID_MTQ', 'ID_HKK']].iloc[:10].reset_index(drop=True)
recommended_MTQ['HKK'] = temp['ID_HKK']
recommended_MTQ['MTQ'] = temp['ID_MTQ']
recommended_MTQ['Rated']= temp['Rating']
recommended_MTQ

,ID_MTQ,Assumed Rating,HKK,MTQ,Rated
0,5900000001,4.609480,00000000011,5900000004,5.0
1,5900000003,4.587755,NaN,NaN,NaN
2,5900000005,4.560517,NaN,NaN,NaN
3,5900000002,3.982890,NaN,NaN,NaN


In [ ]:
user_index = 5
recommended_MTQ = get_recommendation(user_index)
# nhận được những mạnh thường quân được người dùng đánh giá cao khác
temp = data[data['ID_HKK']==df.index[user_index]].sort_values(
    'Rating', ascending=False)[['Rating', 'ID_MTQ', 'ID_HKK']].iloc[:10].reset_index(drop=True)
recommended_MTQ['HKK'] = temp['ID_HKK']
recommended_MTQ['MTQ'] = temp['ID_MTQ']
recommended_MTQ['Rated']= temp['Rating']
recommended_MTQ

,ID_MTQ,Assumed Rating,HKK,MTQ,Rated
0,5900000002,4.982999,00000000006,5900000001,5
1,5900000004,3.969190,00000000006,5900000003,5
